In [3]:
import numpy as np
# import argparse
import imutils
import time
import cv2
import os

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--input", required=True,
#     help="path to input video")
# ap.add_argument("-o", "--output", required=True,
#     help="path to output video")
# ap.add_argument("-y", "--yolo", required=True,
#     help="base path to YOLO directory")
# ap.add_argument("-c", "--confidence", type=float, default=0.5,
#     help="minimum probability to filter weak detections")
# ap.add_argument("-t", "--threshold", type=float, default=0.3,
#     help="threshold when applyong non-maxima suppression")
# args = vars(ap.parse_args())

input_path = "YOLO Object Detection/WIN_20191204_15_22_34_Pro.mp4"
output_path = "output.mp4"
yolo_path = "YOLO Object Detection/yolo-object-detection/yolo-coco"
threshold=0.4
Confidence=0.5

In [4]:
# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([yolo_path, "coco.names"])
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
    dtype="uint8")

# derive the paths to the YOLO weights and model configuration
weightsPath = os.path.sep.join([yolo_path, "yolov3-tiny.weights"])
configPath = os.path.sep.join([yolo_path, "yolov3-tiny.cfg"])

In [5]:
# load our YOLO object detector trained on COCO dataset (80 classes)
# and determine only the *output* layer names that we need from YOLO
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

[INFO] loading YOLO from disk...


In [6]:
# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture(input_path)
writer = None
(W, H) = (None, None)

In [7]:
# try to determine the total number of frames in the video file
try:
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
        else cv2.CAP_PROP_FRAME_COUNT
    total = int(vs.get(prop))
    print("[INFO] {} total frames in video".format(total))

# an error occurred while trying to determine the total
# number of frames in the video file
except:
    print("[INFO] could not determine # of frames in video")
    print("[INFO] no approx. completion time can be provided")
    total = -1

[INFO] 529 total frames in video


In [8]:
# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()

    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break

    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]

    # construct a blob from the input frame and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes
    # and associated probabilities
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    # initialize our lists of detected bounding boxes, confidences,
    # and class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability)
            # of the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > Confidence:
                # scale the bounding box coordinates back relative to
                # the size of the image, keeping in mind that YOLO
                # actually returns the center (x, y)-coordinates of
                # the bounding box followed by the boxes' width and
                # height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top
                # and and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates,
                # confidences, and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping
    # bounding boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, Confidence,
        threshold)

    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indexes we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the frame
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                confidences[i])
            cv2.putText(frame, text, (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_path, fourcc, 30,
            (frame.shape[1], frame.shape[0]), True)

        # some information on processing single frame
    if total > 0:
        elap = (end - start)
        print("[INFO] single frame took {:.4f} seconds".format(elap))
        print("[INFO] estimated total time to finish: {:.4f}".format(
            elap * total))
        
    cv2.imshow("Frames",frame)
    cv2.waitKey(1)

    # write the output frame to disk
    writer.write(frame)

[INFO] single frame took 3.1107 seconds
[INFO] estimated total time to finish: 1645.5527
[INFO] single frame took 0.0559 seconds
[INFO] estimated total time to finish: 29.5450
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0172
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0230
[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4374
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0173
[INFO] single frame took 0.0529 seconds
[INFO] estimated total time to finish: 27.9640
[INFO] single frame took 0.0618 seconds
[INFO] estimated total time to finish: 32.7095
[INFO] single frame took 0.0529 seconds
[INFO] estimated total time to finish: 27.9671
[INFO] single frame took 0.0558 seconds
[INFO] estimated total time to finish: 29.5444
[INFO] single frame took 0.0499 seconds
[INFO] estimated total time to finish: 26.3795
[INFO] single frame took 0.0549 seconds
[

[INFO] single frame took 0.0509 seconds
[INFO] estimated total time to finish: 26.9077
[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4347
[INFO] single frame took 0.0529 seconds
[INFO] estimated total time to finish: 27.9626
[INFO] single frame took 0.0578 seconds
[INFO] estimated total time to finish: 30.5994
[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4347
[INFO] single frame took 0.0509 seconds
[INFO] estimated total time to finish: 26.9070
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0181
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0167
[INFO] single frame took 0.0509 seconds
[INFO] estimated total time to finish: 26.9073
[INFO] single frame took 0.0509 seconds
[INFO] estimated total time to finish: 26.9072
[INFO] single frame took 0.0499 seconds
[INFO] estimated total time to finish: 26.3793
[INFO] single frame took 0.0459 seconds
[IN

[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4374
[INFO] single frame took 0.0509 seconds
[INFO] estimated total time to finish: 26.9063
[INFO] single frame took 0.0529 seconds
[INFO] estimated total time to finish: 27.9626
[INFO] single frame took 0.0618 seconds
[INFO] estimated total time to finish: 32.7111
[INFO] single frame took 0.0529 seconds
[INFO] estimated total time to finish: 27.9613
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0178
[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4341
[INFO] single frame took 0.0588 seconds
[INFO] estimated total time to finish: 31.1283
[INFO] single frame took 0.0509 seconds
[INFO] estimated total time to finish: 26.9070
[INFO] single frame took 0.0559 seconds
[INFO] estimated total time to finish: 29.5452
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0160
[INFO] single frame took 0.0499 seconds
[IN

[INFO] single frame took 0.0459 seconds
[INFO] estimated total time to finish: 24.2694
[INFO] single frame took 0.0439 seconds
[INFO] estimated total time to finish: 23.2142
[INFO] single frame took 0.0479 seconds
[INFO] estimated total time to finish: 25.3248
[INFO] single frame took 0.0459 seconds
[INFO] estimated total time to finish: 24.2694
[INFO] single frame took 0.0459 seconds
[INFO] estimated total time to finish: 24.2660
[INFO] single frame took 0.0449 seconds
[INFO] estimated total time to finish: 23.7416
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0175
[INFO] single frame took 0.0429 seconds
[INFO] estimated total time to finish: 22.6862
[INFO] single frame took 0.0568 seconds
[INFO] estimated total time to finish: 30.0721
[INFO] single frame took 0.0449 seconds
[INFO] estimated total time to finish: 23.7419
[INFO] single frame took 0.0439 seconds
[INFO] estimated total time to finish: 23.2138
[INFO] single frame took 0.0449 seconds
[IN

[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4336
[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4346
[INFO] single frame took 0.0489 seconds
[INFO] estimated total time to finish: 25.8516
[INFO] single frame took 0.0519 seconds
[INFO] estimated total time to finish: 27.4349
[INFO] single frame took 0.0559 seconds
[INFO] estimated total time to finish: 29.5463
[INFO] single frame took 0.0518 seconds
[INFO] estimated total time to finish: 27.4269
[INFO] single frame took 0.0598 seconds
[INFO] estimated total time to finish: 31.6555
[INFO] single frame took 0.0568 seconds
[INFO] estimated total time to finish: 30.0722
[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0181
[INFO] single frame took 0.0509 seconds
[INFO] estimated total time to finish: 26.9064
[INFO] single frame took 0.0529 seconds
[INFO] estimated total time to finish: 27.9621
[INFO] single frame took 0.0549 seconds
[IN

[INFO] single frame took 0.0549 seconds
[INFO] estimated total time to finish: 29.0172
[INFO] single frame took 0.0578 seconds
[INFO] estimated total time to finish: 30.6004
[INFO] single frame took 0.0578 seconds
[INFO] estimated total time to finish: 30.6007
[INFO] single frame took 0.0608 seconds
[INFO] estimated total time to finish: 32.1828
[INFO] single frame took 0.0559 seconds
[INFO] estimated total time to finish: 29.5447
[INFO] single frame took 0.0628 seconds
[INFO] estimated total time to finish: 33.2378
[INFO] single frame took 0.0638 seconds
[INFO] estimated total time to finish: 33.7656
[INFO] single frame took 0.0598 seconds
[INFO] estimated total time to finish: 31.6606
[INFO] single frame took 0.0578 seconds
[INFO] estimated total time to finish: 30.6009
[INFO] single frame took 0.0578 seconds
[INFO] estimated total time to finish: 30.6006
[INFO] single frame took 0.0529 seconds
[INFO] estimated total time to finish: 27.9608
[INFO] single frame took 0.0559 seconds
[IN

In [9]:
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()

[INFO] cleaning up...
